In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
from selenium.webdriver.support.ui import WebDriverWait
import selenium.webdriver.support.expected_conditions as EC
from tqdm.notebook import tqdm
import time


In [2]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")
driver = webdriver.Chrome(executable_path = r'chromedriver.exe',chrome_options = chrome_options)


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_119736\1496906039.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = r'chromedriver.exe',chrome_options = chrome_options)
C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_119736\1496906039.py:3: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path = r'chromedriver.exe',chrome_options = chrome_options)


In [3]:
df1 = pd.read_excel(r"my_sku.xlsx",engine = 'openpyxl')
df1

,asins,Name
0,B09GS5SRVJ,Kinshu
1,B09GS9WKVP,Kinshu
2,B09GT1V57L,Kinshu
3,B09GT4G85M,Kinshu
4,B09GT5L3CC,Kinshu
...,...,...
997,B0B6ZR422H,Kinshu
998,B0B7173P49,Kinshu
999,B0B71H3WCG,Kinshu
1000,B0B746XWVK,Kinshu


In [4]:
df_ASIN = df1["asins"].copy()
df_ASIN.drop_duplicates(inplace=True)
df_ASIN.dropna(inplace = True)
asinList = list(df_ASIN)
len(asinList)

1002

In [5]:
timeout = 10
locator = "imgTagWrapperId"
productDetails = {}
lastIdx = 0
for val in tqdm(range(len(asinList))):
    try:
   
        i = asinList[val]
        print("i",i)
        productDetails[i] = {}
        lastIdx = val
        link = "https:amazon.ae/dp/" + i
        print(link)
        driver.get(link)
        WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.ID, locator)))
        try:
            TitleVal = driver.find_element(By.ID,'title')
            productDetails[i]['Title'] = TitleVal.text
            print(TitleVal.text)
        
        except:
            productDetails[i]['Title'] = "Not Found"
            print("Product Title not found")


        try:
            BrandVal = driver.find_element(By.ID,'bylineInfo')
            productDetails[i]['Brand_Subscript'] = BrandVal.text
            print(BrandVal.text)

        except:
            productDetails[i]['Brand_Subscript'] = "Not Found"
            print("Brand from subscript not found")
            
        try:
            Availval = driver.find_element(By.ID,'availability')
            productDetails[i]['Availability'] = Availval.text
            print(Availval.text)    

        except:
            productDetails[i]['Availability'] = "Not Found"
            print("Brand from subscript not found")    
           
        try:
            Availval = driver.find_element(By.ID,'merchant-info')
            productDetails[i]['Seller'] = Availval.text
            print(Availval.text)    

        except:
            productDetails[i]['Seller'] = "Not Found"
            print("Brand from subscript not found")    
        
        try:
            Table = driver.find_element(By.ID,'productOverview_feature_div')
            df = pd.read_html(Table.get_attribute("innerHTML"))[0]
            productDetails[i].update(dict(zip(df.iloc[:,0],df.iloc[:,1])))
            print("Product Details",productDetails[i])
        except:
            print("No Product Overview Table Found")
            # productDetails[i] = {}
        try:
            productFeatures = driver.find_element(By.ID,'detailBullets_feature_div')
            features = productFeatures.find_elements(By.TAG_NAME,'li')
            featureDetails = {}
            for j in features:
                featurePair = j.text.split(":")
                try:
                    featureDetails[featurePair[0].strip()] = featurePair[1]
                except:
                    pass
            print("featureDetails",featureDetails)
            productDetails[i].update(featureDetails)
            print("Product Details",productDetails[i])
        except:
            print("featureDetails not found")

        try:
            Table2 = driver.find_element(By.ID,'productDetails_techSpec_section_1')
            technicalDetails = {}
            headers = Table2.find_elements(By.TAG_NAME,'th')
            vals = Table2.find_elements(By.TAG_NAME,'td')
            for k in range(len(headers)):
                try:
                    technicalDetails[headers[k].text.strip()] = vals[k].text
                except:
                    pass
            print("technicalDetails",technicalDetails)
            productDetails[i].update(technicalDetails)
            print(productDetails[i])
        except:
            pass
        try:
            Table3 = driver.find_element(By.ID,'productDetails_detailBullets_sections1')
            dfaDetails = {}
            headers = Table3.find_elements(By.TAG_NAME,'th')
            vals = Table3.find_elements(By.TAG_NAME,'td')
            for k in range(len(headers)):
                try:
                    dfaDetails[headers[k].text.strip()] = vals[k].text
                except:
                    pass
            print("dfaDetails",dfaDetails)
            productDetails[i].update(dfaDetails)
            print(productDetails[i])
        except:
            pass
        try:
            
            SRank = driver.find_element(By.ID,'SalesRank')
            BestSellerRankStr = SRank.text.replace("Amazon Bestsellers Rank: ","")
            BestSellerRankDetails = {'Best Sellers Rank': BestSellerRankStr}
            productDetails[i].update(BestSellerRankDetails)
            print(productDetails[i])
        except:
            pass
        
    except:
        pass

  0%|          | 0/1002 [00:00<?, ?it/s]

i B09GS5SRVJ
https:amazon.ae/dp/B09GS5SRVJ
ORiTi 2-Piece Temporary Tattoo Sticker Set Black
Brand: ORiTi
In Stock
Brand from subscript not found
Product Details {'Title': 'ORiTi 2-Piece Temporary Tattoo Sticker Set Black', 'Brand_Subscript': 'Brand: ORiTi', 'Availability': 'In Stock', 'Seller': 'Not Found', 'Brand': 'ORiTi', 'Water resistance level': 'Waterproof', 'Number of pieces': '2'}
featureDetails {'Package Dimensions': ' 9.2 x 6.6 x 0.2 cm; 20 Grams', 'Date First Available': ' 21 September 2021', 'Manufacturer': ' ORiTi', 'ASIN': ' B09GS5SRVJ'}
Product Details {'Title': 'ORiTi 2-Piece Temporary Tattoo Sticker Set Black', 'Brand_Subscript': 'Brand: ORiTi', 'Availability': 'In Stock', 'Seller': 'Not Found', 'Brand': 'ORiTi', 'Water resistance level': 'Waterproof', 'Number of pieces': '2', 'Package Dimensions': ' 9.2 x 6.6 x 0.2 cm; 20 Grams', 'Date First Available': ' 21 September 2021', 'Manufacturer': ' ORiTi', 'ASIN': ' B09GS5SRVJ'}
technicalDetails {'Manufacturer': 'ORiTi', 'P

Lux Botanicals Hand Wash Fig Extract, 250Ml (Pack Of 2), Purple
Brand: Lux
In Stock
Brand from subscript not found
Product Details {'Title': 'Lux Botanicals Hand Wash Fig Extract, 250Ml (Pack Of 2), Purple', 'Brand_Subscript': 'Brand: Lux', 'Availability': 'In Stock', 'Seller': 'Not Found', 'Skin type': 'All', 'Brand': 'Lux', 'Scent': 'Fig', 'Material type free': 'Paraben Free', 'Age range (description)': 'Adult'}
featureDetails {'Package Dimensions': ' 18 x 13.5 x 4.9 cm; 570 Grams', 'Date First Available': ' 21 September 2021', 'Manufacturer': ' Unilever', 'ASIN': ' B09GT5L3CC', 'Country of origin': ' Saudi Arabia'}
Product Details {'Title': 'Lux Botanicals Hand Wash Fig Extract, 250Ml (Pack Of 2), Purple', 'Brand_Subscript': 'Brand: Lux', 'Availability': 'In Stock', 'Seller': 'Not Found', 'Skin type': 'All', 'Brand': 'Lux', 'Scent': 'Fig', 'Material type free': 'Paraben Free', 'Age range (description)': 'Adult', 'Package Dimensions': ' 18 x 13.5 x 4.9 cm; 570 Grams', 'Date First Ava

featureDetails {'Package Dimensions': ' 17.4 x 12.7 x 3.4 cm; 250 Grams', 'Date First Available': ' 23 September 2021', 'Manufacturer': ' GK HAIR', 'ASIN': ' B09GTC3XRT', 'Manufacturer reference': ' GK-MOISTTRI-New', 'Country of origin': ' USA'}
Product Details {'Title': 'GK HAIR Global Keratin Moisturizing 2-in-1 Shampoo and Conditioner Sets Color Protection 100ml with Anti-Frizz Hair Serum 50ml Intense Moisture Pack Trio - For Dry Damaged and Color Treated Hair', 'Brand_Subscript': 'Visit the GK HAIR Store', 'Availability': 'In Stock', 'Seller': 'Not Found', 'Product benefits': 'Anti-Frizz', 'Material type free': 'Ethanol Free, Phthalate Free, Sodium Lauryl Sulphate Free, Paraben Free, Formaldehyde Free', 'Item form': 'Liquid', 'Brand': 'GK HAIR', 'Hair type': 'Dry', 'Package Dimensions': ' 17.4 x 12.7 x 3.4 cm; 250 Grams', 'Date First Available': ' 23 September 2021', 'Manufacturer': ' GK HAIR', 'ASIN': ' B09GTC3XRT', 'Manufacturer reference': ' GK-MOISTTRI-New', 'Country of origin'

featureDetails {'Package Dimensions': ' 21.34 x 14.99 x 8.89 cm; 1.78 Kilograms', 'Date First Available': ' 21 September 2021', 'Manufacturer': ' Unilever', 'ASIN': ' B09GTDK3WW', 'Country of origin': ' United Arab Emirates'}
Product Details {'Title': 'TRESEmmé Keratin Smooth and Straight Shampoo with Argan Oil, 400ml (Pack of 2) + TRESEMMÉ Keratin Smooth and Straight Conditioner, 400ml (Pack of 2)', 'Brand_Subscript': 'Visit the TRESemme Store', 'Availability': 'In Stock', 'Seller': 'Not Found', 'Item form': 'Liquid', 'Brand': 'TRESemme', 'Hair type': 'Frizzy', 'Scent': 'Argan', 'Number of Items': '2', 'Package Dimensions': ' 21.34 x 14.99 x 8.89 cm; 1.78 Kilograms', 'Date First Available': ' 21 September 2021', 'Manufacturer': ' Unilever', 'ASIN': ' B09GTDK3WW', 'Country of origin': ' United Arab Emirates'}
technicalDetails {'Manufacturer': 'Unilever', 'Package Dimensions': '21.34 x 14.99 x 8.89 cm; 1.78 Kilograms', 'ASIN': 'B09GTDK3WW'}
{'Title': 'TRESEmmé Keratin Smooth and Straigh

...

Box of 48 Tattoo Cartridge Needles Holder Container, Disposable Plastic Pin Stand Trays Ink Cups Pigment Rack Holder Tattoo Organiser Plate for Beauticians, Tattoo Artist (Black)
Brand: DERMAPOINT
In Stock
Brand from subscript not found
Product Details {'Title': 'Box of 48 Tattoo Cartridge Needles Holder Container, Disposable Plastic Pin Stand Trays Ink Cups Pigment Rack Holder Tattoo Organiser Plate for Beauticians, Tattoo Artist (Black)', 'Brand_Subscript': 'Brand: DERMAPOINT', 'Availability': 'In Stock', 'Seller': 'Not Found', 'Colour': 'Black', 'Brand': 'DERMAPOINT', 'Number of pieces': '48'}
featureDetails {'Package Dimensions': ' 25.4 x 7.2 x 6.7 cm; 300 Grams', 'Date First Available': ' 30 August 2022', 'Manufacturer': ' Tattoo Cartridges Holder', 'ASIN': ' B0B598YZNH'}
Product Details {'Title': 'Box of 48 Tattoo Cartridge Needles Holder Container, Disposable Plastic Pin Stand Trays Ink Cups Pigment Rack Holder Tattoo Organiser Plate for Beauticians, Tattoo Artist (Black)', 'Bra

dfaDetails {'Best Sellers Rank': '#240,028 in Beauty (See Top 100 in Beauty)\n#59 in Hair Regrowth Devices'}
{'Title': 'Anti Hair Loss Cap,Fajar Anti Hair Loss Cap Hair Growth 94/186/240Pcs Led Infrared Light Against Hair Loss Guard Against The Bald Hair Regeneration Machine Cap', 'Brand_Subscript': 'Brand: FAJAR', 'Availability': "Currently unavailable.\nWe don't know when or if this item will be back in stock.", 'Seller': '', 'Product Dimensions': '28 x 26 x 23 cm', 'Date First Available': ' 29 June 2022', 'Manufacturer': 'FAJAR', 'ASIN': 'B0B5CCG1KV', 'Manufacturer reference': 'YRRW19270W-94TMM', 'Best Sellers Rank': '#240,028 in Beauty (See Top 100 in Beauty)\n#59 in Hair Regrowth Devices'}
i B0B5DJCVLH
https:amazon.ae/dp/B0B5DJCVLH
Unicorn Temporary Tattoos, Unicorn Tattoos Stickers, Fake Girl's Colourful Luminous Temporary Unicorn Tattoo Glow in the Dark for Girl Kid Unicorn Birthday Decoration, 8 Sheets(4 flashes + 4 luminous)
Visit the SYOSI Store
In Stock
Brand from subscript 

Baby Colic and Gas Relief - Heigoeost Heated Tummy Wrap for Newborns Belly Relief by Soothing Warmth, Baby Heating Pad Swaddling Belt Relief & Soothe Gas, Colic and Upset Stomach for Fussy Infants
Brand: Heigoeost
Usually ships within 6 to 7 days
Brand from subscript not found
No Product Overview Table Found
featureDetails {'Package Dimensions': ' 15.19 x 13.41 x 6.5 cm; 259 Grams', 'Manufacturer': ' Heigoeost', 'ASIN': ' B0B5H7M9WS', 'Department': ' unisex-baby'}
Product Details {'Title': 'Baby Colic and Gas Relief - Heigoeost Heated Tummy Wrap for Newborns Belly Relief by Soothing Warmth, Baby Heating Pad Swaddling Belt Relief & Soothe Gas, Colic and Upset Stomach for Fussy Infants', 'Brand_Subscript': 'Brand: Heigoeost', 'Availability': 'Usually ships within 6 to 7 days', 'Seller': 'Not Found', 'Package Dimensions': ' 15.19 x 13.41 x 6.5 cm; 259 Grams', 'Manufacturer': ' Heigoeost', 'ASIN': ' B0B5H7M9WS', 'Department': ' unisex-baby'}
technicalDetails {'Units': '1.0 count', 'Brand':

featureDetails not found
i B0B5VG35ZM
https:amazon.ae/dp/B0B5VG35ZM
i B0B5VGX51Y
https:amazon.ae/dp/B0B5VGX51Y
i B0B5W3RKPB
https:amazon.ae/dp/B0B5W3RKPB
i B0B5WHSHQF
https:amazon.ae/dp/B0B5WHSHQF
i B0B5XGM32Z
https:amazon.ae/dp/B0B5XGM32Z
i B0B5XHWN44
https:amazon.ae/dp/B0B5XHWN44
Nescafe Gold Instant Coffee, 95g
Visit the Nescafe Store
In Stock
Brand from subscript not found
Product Details {'Title': 'Nescafe Gold Instant Coffee, 95g', 'Brand_Subscript': 'Visit the Nescafe Store', 'Availability': 'In Stock', 'Seller': 'Not Found', 'Brand': 'Nescafe', 'Item form': 'Granule', 'Flavour': 'Unflavored', 'Unit count': '95 gram', 'Number of Items': '1'}
featureDetails not found
technicalDetails {'Package Dimensions': '17.8 x 11.4 x 7.5 cm; 95 Grams', 'Units': '95 gram', 'Brand': 'Nescafe', 'Format': 'Granule', 'Special Feature': 'Soluble', 'Caffeine Content': 'Caffeinated', 'Manufacturer': 'Nestle UAE LLC', 'Country of origin': 'Bouvet Island'}
{'Title': 'Nescafe Gold Instant Coffee, 95g', 

i B0B6RT8QK1
https:amazon.ae/dp/B0B6RT8QK1
i B0B6RTNPCZ
https:amazon.ae/dp/B0B6RTNPCZ
i B0B6S52TF6
https:amazon.ae/dp/B0B6S52TF6
i B0B6ST8KL1
https:amazon.ae/dp/B0B6ST8KL1
i B0B6T36NND
https:amazon.ae/dp/B0B6T36NND
i B0B6T7R6KF
https:amazon.ae/dp/B0B6T7R6KF
i B0B6T8GRZ2
https:amazon.ae/dp/B0B6T8GRZ2
Riwayat chicken masala 50g
Brand: Riwayat
In Stock
Brand from subscript not found
Product Details {'Title': 'Riwayat chicken masala 50g', 'Brand_Subscript': 'Brand: Riwayat', 'Availability': 'In Stock', 'Seller': 'Not Found', 'Brand': 'Riwayat', 'Item form': 'Powder', 'Unit count': '50 gram', 'Number of Items': '1', 'Produce sold as': 'Pre-packaged'}
featureDetails not found
i B0B6T94N1H
https:amazon.ae/dp/B0B6T94N1H
i B0B6V6CC93
https:amazon.ae/dp/B0B6V6CC93
Cool & Cool Disinfectant And Sanitizing Multi Purpose Spray 300 Ml X 2 Value Pack
Brand: Cool & Cool
In Stock
Brand from subscript not found
Product Details {'Title': 'Cool & Cool Disinfectant And Sanitizing Multi Purpose Spray 300 Ml 

In [6]:
df_productDetails = pd.DataFrame.from_dict(productDetails,orient='index')

In [7]:
df_productDetails['Brand_Subscript'] = df_productDetails['Brand_Subscript'].str.replace('^Brand:\s','').str.replace('\sStore$','').str.replace('^Visit\sthe\s','')

C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_119736\1461516328.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_productDetails['Brand_Subscript'] = df_productDetails['Brand_Subscript'].str.replace('^Brand:\s','').str.replace('\sStore$','').str.replace('^Visit\sthe\s','')


In [8]:
df_productDetails

,Title,Brand_Subscript,Availability,Seller,Brand,Water resistance level,Number of pieces,Package Dimensions,Date First Available,Manufacturer,...,Theme,Liquid contents description,Is portable,Variety,Liquid flow speed description,Chocolate type,Hand Orientation,Number of Pieces,Features,Percentage assay
B09GS5SRVJ,ORiTi 2-Piece Temporary Tattoo Sticker Set Black,ORiTi,In Stock,Not Found,ORiTi,Waterproof,2,9.2 x 6.6 x 0.2 cm; 20 Grams,21 September 2021,ORiTi,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B09GS9WKVP,bbhugme Pregnancy Body Pillow - Full Body Adju...,bbhugme,Only 3 left in stock (more on the way),Not Found,bbhugme,NaN,NaN,NaN,27 January 2022,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B09GT1V57L,Dove nourishing oil shampoo 400ml + dove nouri...,Dove,In Stock,Not Found,NaN,NaN,NaN,23.5 x 16.2 x 5 cm; 820 Grams,21 September 2021,Unilever,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B09GT4G85M,Lifebuoy Antibacterial Liquid Soap and Hand Wa...,Lifebuoy,In Stock,Not Found,Lifebuoy,NaN,NaN,27.3 x 20.9 x 14.8 cm; 3.11 Kilograms,21 September 2021,Unilever,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B09GT5L3CC,"Lux Botanicals Hand Wash Fig Extract, 250Ml (P...",Lux,In Stock,Not Found,Lux,NaN,NaN,18 x 13.5 x 4.9 cm; 570 Grams,21 September 2021,Unilever,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B0B6N3VB3T,"ONECIRCLE Perfume Atomizer,6ML Refillable Perf...",onecircle,In Stock,Not Found,onecircle,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B0B6T8GRZ2,Riwayat chicken masala 50g,Riwayat,In Stock,Not Found,Riwayat,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B0B6V6CC93,Cool & Cool Disinfectant And Sanitizing Multi ...,Cool & Cool,In Stock,Not Found,Cool & Cool,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B0B6ZN1HJ8,L'Oréal Paris Elvive Hyaluron Moisture Moistur...,L'Oréal Paris,,Not Found,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df_productDetails.to_excel(r"brand NEW.xlsx")